In [ ]:
import pandas as pd
import os
from google.colab import drive 
import numpy as np
import plotly.express as px
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/Shared drives/Capstone Team 15 Drive/Work/Data/')


Mounted at /content/drive


In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import operator
nltk.download('stopwords')
nltk.download('punkt')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
!python -m spacy download en_core_web_md

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import spacy
import string

# Load Data

In [ ]:
df = pd.read_excel('Sample Walmart data.xlsx')
df.head()

,Parent,CausedBy,UponReject,SysUpdatedOn,ApprovalHistory,Skills,Number,URelatedIncidents,USuppressAlert,State,SysCreatedBy,Knowledge,Order,UFunctionLastWorked,UWorkaroundProvidedDtm,UUserUpdated,UWorkaroundProvidedInd,CmdbCi,CmdbCiBusinessApp,Impact,UOnBehalfOf,Active,WorkNotesList,Priority,SysDomainPath,URecordProducer,BusinessDuration,GroupList,UStepsToReplicate,ApprovalSet,ShortDescription,UServiceLevel,CorrelationDisplay,WorkStart,AdditionalAssigneeList,UWorkaround,UOpenedFrom,Notify,ServiceOffering,SysClassName,...,AssignmentGroup,BusinessStc,Description,CalendarDuration,CloseNotes,SysId,ContactType,IncidentState,Urgency,ProblemId,UManualReassignmentCount,Company,UType,ActivityDue,Severity,URequest,UFunctionImpactType,Comments,Approval,DueDate,SysModCount,ReopenCount,SysTags,UEstimatedTaskTime,UCall,UWebhooks,Location,UBusinessImpactStatement,Category,_created_ts,input_dir_name,input_file_name,server,year,month,day,_partitionkey,id,_ts,_uts
0,NaN,NaN,Cancel all future Tasks,2021-01-18T19:09:38.000Z,NaN,NaN,INC10227656,0,False,Assigned,system,False,NaN,NaT,NaN,2019-06-24T19:09:38.000Z,False,oser600497,NaN,2 - Significant,NaN,True,NaN,3 - Medium,/,NaN,NaN,NaN,NaN,NaN,2019-06-24 14:05:17|GLOBAL|SUB:- Alarm:The job...,NaN,NaN,NaN,NaN,NaN,NaN,Do Not Notify,NaN,Incident,...,Automic PROD Support,NaN,2019-06-24 14:05:17|GLOBAL|SUB:- Alarm:The job...,NaN,NaN,9241ccb4dbd673c45191581bdc961940,Alert,Assigned,3 - Medium,NaN,0.0,NaN,NaN,2021-01-25T19:09:38.000Z,3 - Low,NaN,NaN,NaN,Not Yet Requested,NaN,83.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/raw-data/service_now_global_incident/server=W...,20210118_191221_d8290998-0f23-4e73-8ec2-b40b57...,WALMARTGLOBALWORKER_SERVICE-NOW_COM_API,2021.0,1.0,18.0,62.0,INC10227656,2021-01-18T22:09:01.967Z,1.611008e+09
1,NaN,NaN,Cancel all future Tasks,2021-01-17T20:29:03.000Z,NaN,NaN,INC10421105,0,False,Assigned,svcxcall_automation,False,NaN,NaT,NaN,2019-09-29T20:29:03.000Z,False,NaN,NaN,4 - Minor,NaN,True,NaN,4 - Low,/,NaN,NaN,NaN,NaN,NaN,SNAP orders not being dispensed because of equ...,NaN,NaN,NaN,NaN,NaN,NaN,Do Not Notify,NaN,Incident,...,POS CPC DEV,NaN,TRACE Options:---SNAP Order not not found\nIss...,NaN,NaN,41e62a37db9ef7c86fd55c77dc961900,Self-service,Assigned,1 - Critical,NaN,1.0,NaN,NaN,2021-01-24T20:29:03.000Z,3 - Low,NaN,NaN,NaN,Not Yet Requested,NaN,84.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/raw-data/service_now_global_incident/server=W...,20210117_220052_39147758-51ac-4c52-9b49-ba076b...,WALMARTGLOBALWORKER_SERVICE-NOW_COM_API,2021.0,1.0,17.0,52.0,INC10421105,2021-01-17T22:17:26.886Z,1.610922e+09
2,NaN,NaN,Cancel all future Tasks,2021-01-15T10:48:51.000Z,NaN,NaN,INC10431632,0,False,Assigned,r0c01am,False,NaN,NaT,NaN,2019-07-05T10:48:52.000Z,False,NaN,NaN,3 - Moderate,Rahul Chauhan - OM GIF (IDC),True,NaN,3 - Medium,/,NaN,NaN,NaN,NaN,NaN,HO-OLCM QA DB not able to connect.,NaN,NaN,NaN,NaN,NaN,NaN,Do Not Notify,NaN,Incident,...,IF - Foundations,NaN,DB details\ntstr600633.lab.wal-mart.com ...,NaN,NaN,ff7ca608dbe6738c3be95c77dc9619b7,Phone,Assigned,1 - Critical,NaN,2.0,Walmart Technology,NaN,2021-01-22T10:48:51.000Z,3 - Low,NaN,NaN,NaN,Not Yet Requested,NaN,82.0,0.0,NaN,NaN,CALL6820824,NaN,NaN,NaN,NaN,NaN,/raw-data/service_now_global_incident/server=W...,20210116_061858_50e78720-0e78-4be8-83ff-85abdc...,WALMARTGLOBALWORKER_SERVICE-NOW_COM_API,2021.0,1.0,16.0,19.0,INC10431632,2021-01-16T14:40:08.997Z,1.610808e+09
3,NaN,NaN,Cancel all future Tasks,2021-01-15T19:37:15.000Z,NaN,NaN,INC10535181,0,False,Resolved,k6kumar,False,NaN,NaT,NaN,2021-01-15T19:37:15.000Z,False,NaN,NaN,2 - Significant,NaN,True,NaN,4 - Low,/,NaN,132 Days 2 Hours 37 Minutes,NaN,NaN,NaN,FTE field had incorrect values for some had >1...,NaN,NaN,NaN,NaN,NaN,NaN,Do Not Notify,NaN,Incident,...,People Solutions Workday Functional,11414235.0,"FTE field , we are seeing several higher than ...",554 Days 6 Hours 1 Minute,Closing as it's not longer a major issue,d6d79e46dbae3b04cdc832ad7c9619b4,Phone,Resolved,4 - Low,NaN,7.0,INFOSYS LIMITED

# Business Question 1

In [ ]:
df['OpenedAt'] = pd.to_datetime(df['OpenedAt'])
df['day of week'] = df['OpenedAt'].apply(lambda x: x.dayofweek)
df['day of week']
df['hour'] = df['OpenedAt'].apply(lambda x: x.hour)
df['hour']

0       19.0
1       14.0
2        9.0
3       13.0
4        5.0
        ... 
4969     7.0
4970    14.0
4971    14.0
4972    17.0
4973    16.0
Name: hour, Length: 4974, dtype: float64

In [ ]:
temp = df.groupby(['hour','UCiClassName'])['OpenedAt'].count().reset_index()

fig = px.line(temp, x="hour", y="OpenedAt", color='UCiClassName')
fig.show()

In [ ]:
temp2 = df.groupby(['day of week','UCiClassName'])['OpenedAt'].count().reset_index()

fig2 = px.line(temp2, x="day of week", y="OpenedAt", color='UCiClassName')
fig2.show()

In [ ]:
temp3 = df.groupby(['hour','UCiClassName','State'])['State'].count().unstack()
temp3['total'] = temp3.sum(axis=1)
temp3['Resolved_over_total'] = temp3['Resolved']/temp3['total']
temp3

State                                      Assigned  ...  Resolved_over_total
hour UCiClassName                                    ...                     
0.0  Active Directory Domain Controller         6.0  ...                  NaN
     Application                                3.0  ...             0.125000
     Application Environment (Discovered)       1.0  ...                  NaN
     Business Application                       8.0  ...             0.038462
     Business Service                           NaN  ...                  NaN
...                                             ...  ...                  ...
23.0 Thin Client                                NaN  ...             0.500000
     VDI / DDV                                  1.0  ...                  NaN
     VMware vCenter Cluster                     NaN  ...                  NaN
     Windows Server                             NaN  ...                  NaN
     Wireless Access Point                      NaN  ...                  NaN

[903 rows x 8 columns]

In [ ]:
temp3 = df.groupby(['UCiClassName','State'])['State'].count().unstack()

In [ ]:
temp3

State,Assigned,Canceled,Closed,Pending,Resolved,Work in Progress
UCiClassName,,,,,,
Accessory,4.0,NaN,28.0,NaN,3.0,1.0
Active Directory Domain Controller,7.0,NaN,8.0,NaN,1.0,NaN
Active Directory Service,NaN,NaN,4.0,NaN,1.0,NaN
Application,78.0,3.0,242.0,4.0,28.0,4.0
Application Environment (Discovered),8.0,1.0,22.0,1.0,1.0,NaN
...,...,...,...,...,...,...
Video Teleconference Device,NaN,NaN,1.0,NaN,NaN,NaN
Virtualization Server,NaN,NaN,NaN,NaN,NaN,1.0
Windows Server,4.0,2.0,42.0,NaN,NaN,NaN


# Business question 2



In [ ]:
df2 = df[['UCiClassName', 'CommentsAndWorkNotes', 'ShortDescription']].dropna()
df2['TimeStamp'] = df2['CommentsAndWorkNotes'].apply(split1)
df2['CommentsAndWorkNotes'] = df2['CommentsAndWorkNotes'].apply(split2)
df2['cleaned_CAW'] = df2['CommentsAndWorkNotes'].apply(remodel)

In [ ]:
def split1(text):
  try:
    t = text[:19]
    return t
  except TypeError:
    return None

def split2(text):
  try:
    text = text[19:]
    return text
  except TypeError:
    return None

In [ ]:
def costum_tokenize(text):
  try:
    text = re.sub('\W+',' ', text)
    text = word_tokenize(text)
  except TypeError:
    text = 'NaN'
  
  return text

def freq_count(col):
  frq_dic = {}
  for lst in col:
    for i in lst:
      if i not in stop_words and i.isalpha():
        if i in frq_dic:
          frq_dic[i] += 1
        else:
          frq_dic[i] = 1
  return frq_dic

def freq_count2(col):
  frq_dic = {}
  for lst in col:
    for i in lst:
      if i not in stop_words:
        if i in frq_dic:
          frq_dic[i] += 1
        else:
          frq_dic[i] = 1
  return frq_dic

def clean_review(review):
    replace_punctuation = str.maketrans(string.punctuation, ' '*len(string.punctuation))   
    return review.translate(replace_punctuation).lower()

def remodel (text):
  return ' '.join(word_tokenize(clean_review(text)))

# Cosine Similarity

In [ ]:
# function
def cosine_simi (df1):
  simi_df = pd.DataFrame(columns = df1['cleaned'], index = df1['cleaned'])
  lst = list(simi_df.columns)
  lst2 = list(simi_df.index)
  for i in lst:
    for j in lst2:
      documents = [i, j]
      count_vectorizer = CountVectorizer()
      sparse_matrix = count_vectorizer.fit_transform(documents)
      doc_term_matrix = sparse_matrix.todense()
      df = pd.DataFrame(doc_term_matrix, 
                        columns = count_vectorizer.get_feature_names(),
                        index = ['x', 'y'])
      similarity = cosine_similarity(df, df)[0,1]
      simi_df.loc[i,j] = similarity
  
  return simi_df

In [ ]:
lst = list(df2['UCiClassName'].unique())
#lst = lst[4:]
df3 = pd.DataFrame(columns=['ClassName', 'Avg_Spacy_Similarity'])

In [ ]:
for i in lst:
  df4 = df2[df2['UCiClassName'] == i][['UCiClassName','cleaned_CAW']].reset_index().drop(
      columns = ['index']).rename(columns = {'cleaned_CAW':'cleaned'})
  if not df4.empty:
    #df4['cleaned'] = df4['CommentsAndWorkNotes'].apply(remodel)
    simi1 = cosine_simi(df4)
    name = i
    avg = np.mean(np.matrix(simi1))
    df3.loc[len(df3)] =[name, avg] 

In [ ]:
# clustering?
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from scipy.cluster import hierarchy

docs = list(df2['cleaned_CAW'])
#Vectorizing
X = CountVectorizer().fit_transform(docs)
X = TfidfTransformer().fit_transform(X)
#Clustering
X = X.todense()
threshold = 0.1
Z = hierarchy.linkage(X,"average", metric="cosine")
C = hierarchy.fcluster(Z, threshold, criterion="distance")

# Spacy Similarity

In [ ]:
def spacy_simi (df):
  nlp = spacy.load('en_core_web_md')
  simi_df = pd.DataFrame(columns = df['cleaned'], index = df['cleaned'])
  lst = list(simi_df.columns)
  lst2 = list(simi_df.index)
  for i in lst:
    doc1 = nlp(i)
    for j in lst2:
      doc2 = nlp(j)
      simi = doc1.similarity(doc2)
      simi_df.loc[i,j] = simi
  
  return simi_df

In [ ]:
lst = list(df2['UCiClassName'].unique())
df3 = pd.DataFrame(columns=['ClassName', 'Avg_Spacy_Similarity'])

In [ ]:
for i in lst:
  df4 = df2[(df2['UCiClassName'] == i ) & (df2['CommentsAndWorkNotes'].notna())].reset_index().drop(columns = ['index'])
  if not df4.empty:
    df4['cleaned'] = df4['CommentsAndWorkNotes'].apply(remodel)
    simi1 = spacy_simi(df4)
    name = i
    avg = np.mean(np.matrix(simi1))
    df3.loc[len(df3)] =[name, avg] 

In [ ]:
df3

,ClassName,Avg_Spacy_Similarity
0,Application Environment (Discovered),0.895106
1,Linux Server,0.828553
2,Mainframe Job,0.740363


In [ ]:
Batch_job = df2[(df2['UCiClassName'] == 'Linux Server') & (df2['CommentsAndWorkNotes'].notna())].reset_index().drop(columns = ['index'])
Batch_job.shape

(47, 3)

In [ ]:
Batch_job['cleaned'] = Batch_job['CommentsAndWorkNotes'].apply(remodel)

In [ ]:
simi1 = spacy_simi(Batch_job)

In [ ]:
np.mean(np.matrix(simi1))

0.8285528640613093